<a href="https://colab.research.google.com/github/LuisFTacla/WDW_Attendance_Calculator/blob/main/Analise_Rock_n_Roller_Coaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
# <font color=green size=10>ANÁLISE DA ATRAÇÃO ROCK'N'ROLLER COASTER</font>
***

# <font color=green>1 CONHECENDO OS DADOS</font>
***

## <font color=green>1.1 Dataset do projeto</font>
***

# Fonte dos dados: 

https://touringplans.com/blog/2018/06/25/disney-world-wait-times-available-for-data-science-and-machine-learning/



### Variáveis utilizadas

> ### Data e hora
> ***

Instante em que as observações foram realizadas.

> ### Tempo de fila
> ***

Tempo estimado de espera que um visitante que entrasse na fila no dado instante teria de aguardar para andar na atração.

> ### Status
> ***

Status da atração em dado momentoo (aberta (1) / fechada (0)).


#### <font color='red'>Observação</font>
***
> Os dados brutos necessitam dos seguintes tratamento antes de serem analisados:
> 1. Eliminar os registros onde o <b>tempo de fila</b> é inválido (-999);
> 2. Eliminar os registros onde o <b>tempo de fila</b> é NaN;
> 3. Renomear as colunas para 'Data e hora' e 'Tempo de Fila';
> 4. Remover coluna <b>SACTMIN</b> devido a ausência da grande maioria de dados;
> 5. Criar a coluna <b>Status</b> com uma variável booleana que informa se atração está aberta ou fechada.


### Importando bibliotecas e lendo o dataset do projeto

https://pandas.pydata.org/

In [1]:
import pandas as pd
import numpy as np
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

In [2]:
rrc = pd.read_csv('https://raw.githubusercontent.com/LuisFTacla/WDW_Attendance_Calculator/main/Dados/TouringPlans/Rides/Raw_Data/rock_n_rollercoaster.csv')

In [3]:
rrc.head(10)

,date,datetime,SACTMIN,SPOSTMIN
0,01/01/2015,2015-01-01 07:46:22,NaN,5.0
1,01/01/2015,2015-01-01 07:53:19,NaN,5.0
2,01/01/2015,2015-01-01 08:04:29,NaN,20.0
3,01/01/2015,2015-01-01 08:11:19,NaN,20.0
4,01/01/2015,2015-01-01 08:16:29,NaN,20.0
5,01/01/2015,2015-01-01 08:18:22,NaN,20.0
6,01/01/2015,2015-01-01 08:24:23,19.0,NaN
7,01/01/2015,2015-01-01 08:24:37,NaN,20.0
8,01/01/2015,2015-01-01 08:25:19,NaN,20.0
9,01/01/2015,2015-01-01 08:27:37,NaN,30.0


# <font color=green>2 TRATAMENTO DOS DADOS</font>
***

Para o tratamento dos dados, são declaradas funções que recebem o dataframe como parâmetro e, em cada uma, exerce uma modificação específica ao conjunto de dados retornando, por final, o dataframe tratado.

## <font color=green>2.1 Remoção da coluna 'SACTMIN'</font>
***

Coluna 'SACTMIN' possui mais lacunas do que dados úteis, por isso faz-se necessário removê-la.

In [4]:
def remove_sactmin(dados): 
  dados.drop(columns = ['SACTMIN'], inplace = True)
  return dados

## <font color=green>2.2 Remoção de linhas com dados ausentes (NaN) na coluna 'SPOSTMIN'</font>
***


In [5]:
def remove_dados_ausentes(dados):
  dados.dropna(inplace= True)
  return dados

## <font color=green>2.3 Conversão das datas e horas</font>
***

Os dados de 'datetime' estão armazenados como strings. Nesta função, é feita a conversão para timestamps os dados são cpoiados para a coluna recém criada 'data_e_hora'.

In [6]:
def to_datetime(dados):
  dados['data_e_hora'] = pd.to_datetime(dados['datetime'])
  return dados

## <font color=green>2.4 Criação da coluna 'tempo_de_fila' e copiando os dados da coluna 'SPOSTMIN'</font>
***

In [7]:
def cria_coluna_tempo_de_fila(dados):
  dados['tempo_de_fila'] = dados['SPOSTMIN']
  return dados

## <font color=green>2.5 Removendo colunas</font>
***

As colunas 'SPOSTMIN', 'date' e 'datetime' ja possuem seus dados devidamente convertidos e copiados nas colunas criadas, logo, pode-se fazer a remoção.



In [8]:
def remove_colunas(dados):
  dados.drop(columns = ['SPOSTMIN', 'date', 'datetime'], inplace = True)
  return dados

## <font color=green>2.6 Reset Index</font>
***
Com o apagamento de linhas, a numeração do índice contém lacunas.

In [9]:
def reset_index(dados):
  dados.reset_index(drop = True, inplace = True)
  return dados

## <font color=green>2.7 Tratamento</font>
***

Para evitar chamar cada função manualmente, é declarada uma função geral que, dentro dela, são chamadas as funções acima em ordem.

In [10]:
def tratamento_dados(dados):
  remove_sactmin(dados)
  remove_dados_ausentes(dados)
  to_datetime(dados)
  cria_coluna_tempo_de_fila(dados)
  remove_colunas(dados)
  reset_index(dados)

Tratando os dados:

In [11]:
tratamento_dados(rrc)

Visualizando os dados:

In [12]:
rrc.head(10)

,data_e_hora,tempo_de_fila
0,2015-01-01 07:46:22,5.0
1,2015-01-01 07:53:19,5.0
2,2015-01-01 08:04:29,20.0
3,2015-01-01 08:11:19,20.0
4,2015-01-01 08:16:29,20.0
5,2015-01-01 08:18:22,20.0
6,2015-01-01 08:24:37,20.0
7,2015-01-01 08:25:19,20.0
8,2015-01-01 08:27:37,30.0
9,2015-01-01 08:31:20,30.0


## <font color=green>2.8 Filtrando os dados</font>
***

In [13]:
rrc.tempo_de_fila.min()

-999.0

Como podemos ver na célula acima, ainda não terminamos a "limpeza" dos dados! Existem dados inválidos, com valores de -999. Para removê-los, basta aplicarmos um filtro no dataframe:

In [14]:
filtro = rrc.tempo_de_fila >= 0
rrc = rrc[filtro]

In [15]:
rrc.tempo_de_fila.min()

0.0

# <font color=green>3 AJUSTE DOS DADOS</font>
***

Os dados já estão tratados, porém nota-se que as observações foram feitas em horários muito "quebrados". A fim de tornar a plotagem mais esteticamente agradável, iremos ajustar o dataframe para mostrar os dados em instantes de 5 em 5 minutos.


Para fazer esse ajuste, será usada a função de interpolação **interp()** da biblioteca **numpy**. Essa função será chamada da seguinte forma:


> Y_interp = np.interp(X_interp, X, Y)


Sendo:

Y_interp = o resultado final da interpolação, no caso, os dados ajustados;

X_interp = o 'passo' onde os dados serão 'encaixados', ou seja, os instantes listados de 5 em 5 minutos;

X = os instantes das observações, em horários não ajustados;

Y = os dados de tempos de fila.


Antes de realizar a interpolação, precisamos fazer duas coisas: criar um dataframe de 'base' em que listaremos os instantes de 5 em 5 minutos, o qual usaremos na interpolação; unir a base com os dados, reordenar de acordo com a data e filtrar este novo dataframe. 

#### <font color='red'>Observação</font>
***
> Este último passo se faz necessário pois, como os parques não operam durante a madrugada, não há observações nos dados brutos. O problema seria visto durante a plotagem dos gráficos diários. Como a última observação de um dia i e a primeira do dia i+1 não são iguais a zero, a função interp iria atribuir valores diferentes de zero nos horários de parque fechado.

> Exemplificando: supondo que no dia 25, o último dado foi obtido às 23h37, com a fila tendo 5 minutos de duração. No dia 26, por sua vez, o primeiro dado, obtido às 7h49, mostra que a fila está com 10 minutos. A função interp iria atribuir a todo esse intervalo valores crescentes entre 5 e 10 minutos, como se houvesse alguma fila durante a madrugada, o que não condiz com a realidade.

## <font color=green>3.1 Criando dataframe 'base'</font>
***

#### <font color='red'>Observação</font>
***
> O número de observações do dataframe base é obtido com os seguintes dados:

365 dias por ano;

5 anos de dados (2015 a 2019);

1 dia a mais por ano bissexto (2016);

24 horas por dia;

12 intervalos de 5 minutos dentro de 1 hora.

In [16]:
# Calculando numero de observacoes
n_observacoes = ((365 * 5) + 1) * 24 * 12


# Criação de um índice de datas/horas
instantes_5_em_5 = pd.date_range(start='1/1/2015', # Data de início da série
                                 periods = n_observacoes, # 
                                 freq = '5min')

# Criação do dataframe de base
base = pd.DataFrame(index = range(0,n_observacoes), columns = ['data_e_hora', 
                                                               'tempo_de_fila'])
base['data_e_hora'] = instantes_5_em_5
base['tempo_de_fila'] = 0

In [17]:
base.head(10)

,data_e_hora,tempo_de_fila
0,2015-01-01 00:00:00,0
1,2015-01-01 00:05:00,0
2,2015-01-01 00:10:00,0
3,2015-01-01 00:15:00,0
4,2015-01-01 00:20:00,0
5,2015-01-01 00:25:00,0
6,2015-01-01 00:30:00,0
7,2015-01-01 00:35:00,0
8,2015-01-01 00:40:00,0
9,2015-01-01 00:45:00,0


## <font color=green>3.2 Unindo dataframes base + dados</font>
***

In [18]:
rrc_df = base.append(rrc)

In [19]:
rrc_df

,data_e_hora,tempo_de_fila
0,2015-01-01 00:00:00,0.0
1,2015-01-01 00:05:00,0.0
2,2015-01-01 00:10:00,0.0
3,2015-01-01 00:15:00,0.0
4,2015-01-01 00:20:00,0.0
...,...,...
211432,2019-12-31 22:58:02,45.0
211433,2019-12-31 23:09:02,35.0
211434,2019-12-31 23:16:02,35.0
211435,2019-12-31 23:23:02,35.0


## <font color=green>3.3 Ordenando dados pela data e hora</font>
***

In [20]:
rrc_df.sort_values(by = ['data_e_hora'], inplace = True)

In [21]:
rrc_df

,data_e_hora,tempo_de_fila
0,2015-01-01 00:00:00,0.0
1,2015-01-01 00:05:00,0.0
2,2015-01-01 00:10:00,0.0
3,2015-01-01 00:15:00,0.0
4,2015-01-01 00:20:00,0.0
...,...,...
525883,2019-12-31 23:35:00,0.0
525884,2019-12-31 23:40:00,0.0
525885,2019-12-31 23:45:00,0.0
525886,2019-12-31 23:50:00,0.0


## <font color=green>3.4 Reset Index</font>
***

In [22]:
reset_index(rrc_df)

,data_e_hora,tempo_de_fila
0,2015-01-01 00:00:00,0.0
1,2015-01-01 00:05:00,0.0
2,2015-01-01 00:10:00,0.0
3,2015-01-01 00:15:00,0.0
4,2015-01-01 00:20:00,0.0
...,...,...
725259,2019-12-31 23:35:00,0.0
725260,2019-12-31 23:40:00,0.0
725261,2019-12-31 23:45:00,0.0
725262,2019-12-31 23:50:00,0.0


## <font color=green>3.5 Apagando linhas</font>
***

#### <font color='red'>Observação</font>
***
> Esse passo é necessário pois ao misturar os dataframes base e dados, em meio aos dados, existem algumas linhas com valores iguais a zero, vindas do dataframe de base. Neste passo, iremos criar uma coluna 'apagar' que receberá valores booleanos. Se o campo em que o algoritmo analisar cumprir uma das condições, este será deletado.

In [23]:
rrc_df['apagar'] = 0

In [24]:
rrc_df

,data_e_hora,tempo_de_fila,apagar
0,2015-01-01 00:00:00,0.0,0
1,2015-01-01 00:05:00,0.0,0
2,2015-01-01 00:10:00,0.0,0
3,2015-01-01 00:15:00,0.0,0
4,2015-01-01 00:20:00,0.0,0
...,...,...,...
725259,2019-12-31 23:35:00,0.0,0
725260,2019-12-31 23:40:00,0.0,0
725261,2019-12-31 23:45:00,0.0,0
725262,2019-12-31 23:50:00,0.0,0


A condição para uma linha ser deletada é se, primeiro, o valor do tempo de fila dela for igual à zero, e se um dos dois valores seguintes ou dois valores anteriores de tempos de fila for diferente de zero. Ou seja, caso os quatro sejam iguais à zero, a linha permanece. Mas se apenas um deles for diferente de zero, a linha é deletada.

In [25]:
for i in range(0,len(rrc_df)):
    if 1 < i < len(rrc_df) - 2:
      if (rrc_df.tempo_de_fila[i] == 0):
        c1 = (rrc_df.tempo_de_fila[i + 1] != 0)
        c2 = (rrc_df.tempo_de_fila[i - 1] != 0)
        c3 = (rrc_df.tempo_de_fila[i + 2] != 0)
        c4 = (rrc_df.tempo_de_fila[i - 2] != 0)
        if c1 or c2 or c3 or c4:
          rrc_df.apagar[i] = 1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Essa seleção pode ser notada logo no final da série de dados:

In [27]:
rrc_df.tail(10)

,data_e_hora,tempo_de_fila,apagar
725254,2019-12-31 23:20:00,0.0,1
725255,2019-12-31 23:23:02,35.0,0
725256,2019-12-31 23:25:00,0.0,1
725257,2019-12-31 23:30:00,0.0,1
725258,2019-12-31 23:30:02,35.0,0
725259,2019-12-31 23:35:00,0.0,1
725260,2019-12-31 23:40:00,0.0,1
725261,2019-12-31 23:45:00,0.0,0
725262,2019-12-31 23:50:00,0.0,0
725263,2019-12-31 23:55:00,0.0,0


Feita a seleção, agora basta apenas filtrar o dataframe.

In [28]:
filtro2 = rrc_df.apagar == 0 
rrc_df = rrc_df[filtro2]

Podemos notar que o filtro foi aplicado com sucesso:

In [29]:
rrc_df.tail(10)

,data_e_hora,tempo_de_fila,apagar
725242,2019-12-31 22:44:01,60.0,0
725245,2019-12-31 22:51:02,45.0,0
725247,2019-12-31 22:58:02,45.0,0
725250,2019-12-31 23:09:02,35.0,0
725253,2019-12-31 23:16:02,35.0,0
725255,2019-12-31 23:23:02,35.0,0
725258,2019-12-31 23:30:02,35.0,0
725261,2019-12-31 23:45:00,0.0,0
725262,2019-12-31 23:50:00,0.0,0
725263,2019-12-31 23:55:00,0.0,0


## <font color=green>3.6 Reset Index e deletando a coluna 'apagar'</font>
***

In [30]:
reset_index(rrc_df)

,data_e_hora,tempo_de_fila,apagar
0,2015-01-01 00:00:00,0.0,0
1,2015-01-01 00:05:00,0.0,0
2,2015-01-01 00:10:00,0.0,0
3,2015-01-01 00:15:00,0.0,0
4,2015-01-01 00:20:00,0.0,0
...,...,...,...
455491,2019-12-31 23:23:02,35.0,0
455492,2019-12-31 23:30:02,35.0,0
455493,2019-12-31 23:45:00,0.0,0
455494,2019-12-31 23:50:00,0.0,0


In [31]:
rrc_df.drop(columns = ['apagar'], inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [32]:
rrc_df

,data_e_hora,tempo_de_fila
0,2015-01-01 00:00:00,0.0
1,2015-01-01 00:05:00,0.0
2,2015-01-01 00:10:00,0.0
3,2015-01-01 00:15:00,0.0
4,2015-01-01 00:20:00,0.0
...,...,...
455491,2019-12-31 23:23:02,35.0
455492,2019-12-31 23:30:02,35.0
455493,2019-12-31 23:45:00,0.0
455494,2019-12-31 23:50:00,0.0


## <font color=green>3.7 Interpolação dos tempos de fila</font>
***

In [33]:
X_interp = base.data_e_hora
X = rrc_df.data_e_hora
Y = rrc_df.tempo_de_fila

Y_interp = np.interp(X_interp, X, Y)

## <font color=green>3.8 Criação do dataframe final</font>
***

In [37]:
RocknRollerCoaster = pd.DataFrame(index = range(0,n_observacoes), 
                                  columns = ['data_e_hora', 'tempo_de_fila'])
RocknRollerCoaster['data_e_hora'] = instantes_5_em_5
RocknRollerCoaster['tempo_de_fila'] = np.around(Y_interp)

In [38]:
RocknRollerCoaster

,data_e_hora,tempo_de_fila
0,2015-01-01 00:00:00,0.0
1,2015-01-01 00:05:00,0.0
2,2015-01-01 00:10:00,0.0
3,2015-01-01 00:15:00,0.0
4,2015-01-01 00:20:00,0.0
...,...,...
525883,2019-12-31 23:35:00,23.0
525884,2019-12-31 23:40:00,12.0
525885,2019-12-31 23:45:00,0.0
525886,2019-12-31 23:50:00,0.0
